In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

!pip install --quiet torch torchvision webdataset tqdm pillow scikit-learn joblib


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path
import sys, yaml, torch, importlib, logging, joblib
from collections import defaultdict, Counter
from tqdm import tqdm
import numpy as np

# 📁 Configurazione percorso progetto
config_path = Path('/content/drive/MyDrive/Colab Notebooks/MLA_PROJECT/wsi-ssrl-rcc_project/config/training.yaml')
with config_path.open('r') as f:
    cfg = yaml.safe_load(f)

colab_root = Path(cfg['env_paths']['colab'])
local_root = Path(cfg['env_paths']['local'])
PROJECT_ROOT = colab_root if colab_root.exists() else local_root

sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

# Normalizza i path dei dati
for split in ['train','val','test']:
    rel = cfg['data'].get(split)
    if rel:
        cfg['data'][split] = str(PROJECT_ROOT / rel)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("EVAL")


In [3]:
# 📁 Setup esperimento specifico
experiment_dir = PROJECT_ROOT / "data/processed2/dataset_9f30917e/experiments/20250624222813"
results_dir = experiment_dir / "results"
results_dir.mkdir(parents=True, exist_ok=True)

results_file = results_dir / "evaluation_summary.csv"
per_patient_file = results_dir / "per_patient_predictions.csv"

# Inizializza CSV se non esistono
import csv
if not results_file.exists():
    with open(results_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Model", "Accuracy", "Macro F1", "Macro Precision", "Macro Recall", "N_Patients"])
if not per_patient_file.exists():
    with open(per_patient_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Model", "Patient_ID", "True_Label", "Predicted_Label"])

# 🔁 Imposta i modelli da valutare
run_model = cfg.get("run_model", "all").lower()
models_cfg = cfg["models"]
tasks = list(models_cfg.items()) if run_model == "all" else [(run_model, models_cfg[run_model])]


In [4]:
from importlib.util import spec_from_file_location, module_from_spec

spec = spec_from_file_location("training_utils", str(PROJECT_ROOT / "src/utils/training_utils.py"))
training_utils = module_from_spec(spec)
spec.loader.exec_module(training_utils)
sys.modules["utils.training_utils"] = training_utils

from utils.training_utils import TRAINER_REGISTRY, load_checkpoint
from trainers.extract_features import extract_features

trainer_modules = [
    "trainers.simclr",
    "trainers.moco_v2",
    "trainers.rotation",
    "trainers.jigsaw",
    "trainers.supervised",
    "trainers.transfer",
]
for m in trainer_modules:
    if m in sys.modules:
        importlib.reload(sys.modules[m])
    else:
        importlib.import_module(m)


In [5]:
def majority_vote(preds, keys):
    patient_to_preds = defaultdict(list)
    for key, pred in zip(keys, preds):
        parts = key.split("_")
        patient_id = next((p for p in parts if p.startswith("HP")), "unknown")
        patient_to_preds[patient_id].append(pred)

    voted_labels = []
    true_labels = []
    for patient_id, votes in patient_to_preds.items():
        counter = Counter(votes)
        majority_label = counter.most_common(1)[0][0]
        voted_labels.append(majority_label)
    return voted_labels


In [6]:
def evaluate_selfsupervised(trainer, classifier_path, test_path, save_dir=None):
    import webdataset as wds
    import torchvision.transforms as T
    from PIL import Image
    from collections import defaultdict, Counter
    import torch
    import joblib
    from sklearn.metrics import classification_report, confusion_matrix

    logger.info("🧪 Evaluation (Self-Supervised)")

    # Carica classificatore e encoder
    model = joblib.load(classifier_path)
    clf = model["model"]
    le = model["label_encoder"]

    def make_test_loader():
        ds = (
            wds.WebDataset(test_path, shardshuffle=False, handler=wds.warn_and_continue, empty_check=False)
            .decode("pil")
            .map(lambda sample: {
                "img": T.ToTensor()(
                    next((v for k, v in sample.items() if isinstance(v, Image.Image)), None).convert("RGB")
                ),
                "key": sample["__key__"] + "." + next((k for k in sample.keys() if k.endswith(".jpg")), "")
            })
        )
        return torch.utils.data.DataLoader(
            ds,
            batch_size=64,
            shuffle=False,
            num_workers=0,
            pin_memory=True
        )

    # Estrai feature
    loader = make_test_loader()
    # ↪️ Scegli modello corretto per feature extraction
    if hasattr(trainer, "model"):
        feature_model = trainer.model
    elif hasattr(trainer, "encoder"):
        feature_model = trainer.encoder
    else:
        raise AttributeError(f"❌ Trainer {trainer.__class__.__name__} non ha né model né encoder per extract_features.")

    feats = extract_features(feature_model, loader, trainer.device)

    X = feats["features"].numpy()
    keys = feats["keys"]
    print(f"Keys trovate----->>>> {keys}")
    y_pred = clf.predict(X)

    def extract_patient_id(k):
        parts = k.split("_")
        for part in parts:
            if part.startswith("HP") or part.startswith("H"):
                return part
        return "UNKNOWN"

    def extract_label_from_key(k):
        if k.startswith("not_tumor"):
            return "not_tumor"
        return k.split("_")[0]

    # Step 1: Costruisci dizionario paziente → label tumorale GT (escludendo not_tumor)
    true_patient_labels = {}
    all_labels_per_patient = defaultdict(list)

    for k in keys:
        label = extract_label_from_key(k)
        pid = extract_patient_id(k)
        all_labels_per_patient[pid].append(label)

    for pid, labels in all_labels_per_patient.items():
        tumor_labels = [lab for lab in labels if lab != "not_tumor"]
        if len(set(tumor_labels)) == 1:
            true_patient_labels[pid] = tumor_labels[0]
        elif len(set(tumor_labels)) > 1:
            logger.warning(f"⚠️ Paziente {pid} ha più classi tumorali: {set(tumor_labels)}. Skippato.")
    print(f"TRUE LABEL PER PAZIENTE {true_patient_labels}")

    # Step 2: Costruisci predizioni per paziente (escludendo predizioni not_tumor)
    preds_per_patient = defaultdict(list)
    for k, pred in zip(keys, y_pred):
        pid = extract_patient_id(k)
        if le.classes_[pred] != "not_tumor":
            preds_per_patient[pid].append(pred)

    # Step 3: Majority voting per pazienti validi
    y_true, y_majority, valid_pids = [], [], []
    for pid, preds in preds_per_patient.items():
        if pid not in true_patient_labels or len(preds) == 0:
            continue
        gt_label = true_patient_labels[pid]
        majority = Counter(preds).most_common(1)[0][0]
        y_true.append(le.transform([gt_label])[0])
        y_majority.append(majority)
        valid_pids.append(pid)




    if len(y_true) == 0 or len(y_majority) == 0:
        print("❌ Nessun paziente valutabile: majority voting vuoto o label GT non disponibili.")
        return

    print("\n📊 Risultati Majority Voting (paziente-level):")
    print(classification_report(y_true, y_majority, target_names=[c for c in le.classes_ if c != "not_tumor"]))
    print("📉 Confusion Matrix:")
    print(confusion_matrix(y_true, y_majority))
    print(f"✅ Totale pazienti classificati: {len(y_true)}")

    print("\n🧾 Predizione per paziente:")
    for pid, true_encoded, pred_encoded in zip(valid_pids, y_true, y_majority):
        true_label = le.inverse_transform([true_encoded])[0]
        pred_label = le.inverse_transform([pred_encoded])[0]
        print(f"• Paziente {pid}: predetto = {pred_label} | reale = {true_label}")


    from sklearn.metrics import f1_score, precision_score, recall_score

    # Calcolo metriche
    acc = np.mean(np.array(y_true) == np.array(y_majority))
    macro_f1 = f1_score(y_true, y_majority, average="macro")
    macro_precision = precision_score(y_true, y_majority, average="macro")
    macro_recall = recall_score(y_true, y_majority, average="macro")
    target_results_file = save_dir / "evaluation_summary.csv" if save_dir else results_file
    with open(target_results_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["selfsupervised_" + trainer.__class__.__name__, acc, macro_f1, macro_precision, macro_recall, len(y_true)])

    target_per_patient_file = save_dir / "per_patient_predictions.csv" if save_dir else per_patient_file
    with open(target_per_patient_file, "a", newline="") as f:
        writer = csv.writer(f)
        for pid, true_encoded, pred_encoded in zip(valid_pids, y_true, y_majority):
            true_label = le.inverse_transform([true_encoded])[0]
            pred_label = le.inverse_transform([pred_encoded])[0]
            writer.writerow(["selfsupervised_" + trainer.__class__.__name__, pid, true_label, pred_label])



In [7]:
def evaluate_supervised(trainer, test_path, save_dir=None):
    import webdataset as wds
    import torchvision.transforms as T
    from PIL import Image
    from sklearn.metrics import classification_report, confusion_matrix

    logger.info("🧪 Evaluation (Supervised/Transfer con Majority Voting)")

    def extract_patient_id(k):
        parts = k.split("_")
        for part in parts:
            if part.startswith("HP") or part.startswith("H"):
                return part
        return "UNKNOWN"

    def extract_label_from_key(k):
        if k.startswith("not_tumor"):
            return "not_tumor"
        return k.split("_")[0]

    def make_test_loader():
        ds = (
            wds.WebDataset(test_path, shardshuffle=False, handler=wds.warn_and_continue, empty_check=False)
            .decode("pil")
            .map(lambda sample: {
                "img": T.ToTensor()(
                    next((v for k, v in sample.items() if isinstance(v, Image.Image)), None).convert("RGB")
                ),
                "key": sample["__key__"] + "." + next((k for k in sample.keys() if k.endswith(".jpg")), "")
            })
        )
        return torch.utils.data.DataLoader(ds, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

    # Step 1: Estrai feature e predizioni
    loader = make_test_loader()
    model = trainer.model.to(trainer.device)
    model.eval()
    le = trainer.label_encoder

    y_pred, keys = [], []

    with torch.no_grad():
        for batch in loader:
            imgs = batch["img"].to(trainer.device)
            logits = model(imgs)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            y_pred.extend(preds)
            keys.extend(batch["key"])

    # Step 2: Costruisci mappa paziente → label GT tumorale
    true_patient_labels = {}
    all_labels_per_patient = defaultdict(list)
    for k in keys:
        label = extract_label_from_key(k)
        pid = extract_patient_id(k)
        all_labels_per_patient[pid].append(label)

    for pid, labels in all_labels_per_patient.items():
        tumor_labels = [lab for lab in labels if lab != "not_tumor"]
        if len(set(tumor_labels)) == 1:
            true_patient_labels[pid] = tumor_labels[0]
        elif len(set(tumor_labels)) > 1:
            logger.warning(f"⚠️ Paziente {pid} ha più classi tumorali: {set(tumor_labels)}. Skippato.")
    print(f"TRUE LABEL PER PAZIENTE {true_patient_labels}")

    # Step 3: Raggruppa predizioni per paziente (escludi not_tumor)
    preds_per_patient = defaultdict(list)
    for k, pred in zip(keys, y_pred):
        pid = extract_patient_id(k)
        if le.classes_[pred] != "not_tumor":
            preds_per_patient[pid].append(pred)

    # Step 4: Majority voting
    y_true, y_majority, valid_pids = [], [], []
    for pid, preds in preds_per_patient.items():
        if pid not in true_patient_labels or len(preds) == 0:
            continue
        gt_label = true_patient_labels[pid]
        majority = Counter(preds).most_common(1)[0][0]
        y_true.append(le.transform([gt_label])[0])
        y_majority.append(majority)
        valid_pids.append(pid)

    if len(y_true) == 0 or len(y_majority) == 0:
        print("❌ Nessun paziente valutabile.")
        return

    print("\n📊 Risultati Majority Voting (paziente-level):")
    print(classification_report(y_true, y_majority, target_names=[c for c in le.classes_ if c != "not_tumor"]))
    print("📉 Confusion Matrix:")
    print(confusion_matrix(y_true, y_majority))
    print(f"✅ Totale pazienti classificati: {len(y_true)}")

    print("\n🧾 Predizione per paziente:")
    for pid, true_encoded, pred_encoded in zip(valid_pids, y_true, y_majority):
        true_label = le.inverse_transform([true_encoded])[0]
        pred_label = le.inverse_transform([pred_encoded])[0]
        print(f"• Paziente {pid}: predetto = {pred_label} | reale = {true_label}")
    from sklearn.metrics import f1_score, precision_score, recall_score

    acc = np.mean(np.array(y_true) == np.array(y_majority))
    macro_f1 = f1_score(y_true, y_majority, average="macro")
    macro_precision = precision_score(y_true, y_majority, average="macro")
    macro_recall = recall_score(y_true, y_majority, average="macro")

    target_results_file = save_dir / "evaluation_summary.csv" if save_dir else results_file
    with open(target_results_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["supervised_" + trainer.__class__.__name__, acc, macro_f1, macro_precision, macro_recall, len(y_true)])

    target_per_patient_file = save_dir / "per_patient_predictions.csv" if save_dir else per_patient_file
    with open(target_per_patient_file, "a", newline="") as f:
        writer = csv.writer(f)
        for pid, true_encoded, pred_encoded in zip(valid_pids, y_true, y_majority):
            true_label = le.inverse_transform([true_encoded])[0]
            pred_label = le.inverse_transform([pred_encoded])[0]
            writer.writerow(["supervised_" + trainer.__class__.__name__, pid, true_label, pred_label])




In [8]:
# Percorso esperimento corretto per tutti i modelli
experiment_dir = PROJECT_ROOT / "data/processed2/dataset_9f30917e/experiments/20250624222813"

for name, m_cfg in tasks:
    if name not in TRAINER_REGISTRY:
        logger.warning(f"❌ Trainer '{name}' non trovato.")
        continue

    trainer = TRAINER_REGISTRY[name](m_cfg, cfg["data"])
    test_path = str(cfg["data"]["test"])
    model_dir = experiment_dir / name

    if not model_dir.exists():
        logger.warning(f"⚠️ Directory modello '{name}' non trovata in {experiment_dir}, skipping.")
        continue

    # 📌 Caso supervised e transfer
    if name in ["supervised", "transfer"]:
      ckpt_list = list(model_dir.glob(f"*Trainer_best_epoch*.pt"))  # ✅ più robusto
      if not ckpt_list:
          logger.warning(f"⚠️ Checkpoint non trovato per '{name}', skipping.")
          continue
      ckpt = ckpt_list[-1]
      load_checkpoint(ckpt, model=trainer.model)
      trainer.model = trainer.model.to(trainer.device)
      evaluate_supervised(trainer, test_path, save_dir=model_dir)
      continue


    # 📌 Caso self-supervised (SimCLR, MoCo, Jigsaw, Rotation)
    classifier_path = model_dir / f"{name}_classifier.joblib"
    features_path   = model_dir / f"{name}_features.pt"
    ckpt_list = list(model_dir.glob(f"*Trainer_best_epoch*.pt"))

    if not classifier_path.exists() or not features_path.exists() or not ckpt_list:
        logger.warning(f"⚠️ File mancanti per '{name}' (classifier, features o checkpoint), skipping.")
        continue

    ckpt = ckpt_list[-1]

    try:
        # Primo tentativo: modello = encoder + projector
        model = torch.nn.Sequential(trainer.encoder, trainer.projector)
        load_checkpoint(ckpt, model=model)
        trainer.encoder = model[0].to(trainer.device)
        trainer.projector = model[1].to(trainer.device)
    except Exception as e1:
        try:
            # Secondo tentativo: solo model
            load_checkpoint(ckpt, model=trainer.model)
            trainer.model = trainer.model.to(trainer.device)
        except Exception as e2:
            try:
                # Terzo tentativo: encoder + head
                model = torch.nn.Sequential(trainer.encoder, trainer.head)
                load_checkpoint(ckpt, model=model)
                trainer.encoder = model[0].to(trainer.device)
                trainer.head = model[1].to(trainer.device)
            except Exception as e3:
                raise RuntimeError(
                    f"❌ Impossibile caricare checkpoint per '{name}'. Tentativi falliti:\n"
                    f"  encoder+projector: {e1}\n"
                    f"  model: {e2}\n"
                    f"  encoder+head: {e3}"
                )

    evaluate_selfsupervised(trainer, classifier_path, test_path, save_dir=model_dir)


/usr/local/lib/python3.11/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Extracting features: 8it [00:56,  7.00s/it]


Keys trovate----->>>> ['ONCO_HP20002450_000002.', 'CHROMO_HP18014084_000007.', 'ONCO_HP20002450_000009.', 'pRCC_HP12.7726_000012.jpg', 'CHROMO_HP18014084_000035.', 'CHROMO_HP18014084_000037.', 'CHROMO_HP18014084_000038.', 'ONCO_HP20002450_000044.', 'ccRCC_HP11.12318_000048.jpg', 'CHROMO_HP18014084_000049.', 'pRCC_HP19.1277_000062.jpg', 'CHROMO_HP18014084_000063.', 'ONCO_HP20002450_000070.', 'CHROMO_HP18014084_000076.', 'not_tumor_HP12.13358_000078.jpg', 'ONCO_HP20002450_000082.', 'ONCO_HP20002450_000083.', 'CHROMO_HP18014084_000084.', 'ccRCC_HP19.4075_000085.jpg', 'pRCC_HP18.5818_000096.jpg', 'pRCC_HP14.4279_000103.jpg', 'not_tumor_HP12.13358_000112.jpg', 'CHROMO_HP18014084_000113.', 'ONCO_HP20002450_000117.', 'not_tumor_HP12.13588_000123.jpg', 'ONCO_HP20002450_000124.', 'not_tumor_HP18.13618_000125.jpg', 'CHROMO_HP18014084_000126.', 'ONCO_HP20002450_000129.', 'ONCO_HP20002450_000132.', 'not_tumor_HP19.4075_000136.jpg', 'pRCC_HP18.5818_000141.jpg', 'ccRCC_HP12.13588_000145.jpg', 'not_t

/usr/local/lib/python3.11/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Extracting features: 8it [00:48,  6.06s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and bein

Keys trovate----->>>> ['ONCO_HP20002450_000002.', 'CHROMO_HP18014084_000007.', 'ONCO_HP20002450_000009.', 'pRCC_HP12.7726_000012.jpg', 'CHROMO_HP18014084_000035.', 'CHROMO_HP18014084_000037.', 'CHROMO_HP18014084_000038.', 'ONCO_HP20002450_000044.', 'ccRCC_HP11.12318_000048.jpg', 'CHROMO_HP18014084_000049.', 'pRCC_HP19.1277_000062.jpg', 'CHROMO_HP18014084_000063.', 'ONCO_HP20002450_000070.', 'CHROMO_HP18014084_000076.', 'not_tumor_HP12.13358_000078.jpg', 'ONCO_HP20002450_000082.', 'ONCO_HP20002450_000083.', 'CHROMO_HP18014084_000084.', 'ccRCC_HP19.4075_000085.jpg', 'pRCC_HP18.5818_000096.jpg', 'pRCC_HP14.4279_000103.jpg', 'not_tumor_HP12.13358_000112.jpg', 'CHROMO_HP18014084_000113.', 'ONCO_HP20002450_000117.', 'not_tumor_HP12.13588_000123.jpg', 'ONCO_HP20002450_000124.', 'not_tumor_HP18.13618_000125.jpg', 'CHROMO_HP18014084_000126.', 'ONCO_HP20002450_000129.', 'ONCO_HP20002450_000132.', 'not_tumor_HP19.4075_000136.jpg', 'pRCC_HP18.5818_000141.jpg', 'ccRCC_HP12.13588_000145.jpg', 'not_t

Extracting features: 8it [00:48,  6.00s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/

...

/usr/local/lib/python3.11/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Extracting features: 8it [00:58,  7.33s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and bein

Keys trovate----->>>> ['ONCO_HP20002450_000002.', 'CHROMO_HP18014084_000007.', 'ONCO_HP20002450_000009.', 'pRCC_HP12.7726_000012.jpg', 'CHROMO_HP18014084_000035.', 'CHROMO_HP18014084_000037.', 'CHROMO_HP18014084_000038.', 'ONCO_HP20002450_000044.', 'ccRCC_HP11.12318_000048.jpg', 'CHROMO_HP18014084_000049.', 'pRCC_HP19.1277_000062.jpg', 'CHROMO_HP18014084_000063.', 'ONCO_HP20002450_000070.', 'CHROMO_HP18014084_000076.', 'not_tumor_HP12.13358_000078.jpg', 'ONCO_HP20002450_000082.', 'ONCO_HP20002450_000083.', 'CHROMO_HP18014084_000084.', 'ccRCC_HP19.4075_000085.jpg', 'pRCC_HP18.5818_000096.jpg', 'pRCC_HP14.4279_000103.jpg', 'not_tumor_HP12.13358_000112.jpg', 'CHROMO_HP18014084_000113.', 'ONCO_HP20002450_000117.', 'not_tumor_HP12.13588_000123.jpg', 'ONCO_HP20002450_000124.', 'not_tumor_HP18.13618_000125.jpg', 'CHROMO_HP18014084_000126.', 'ONCO_HP20002450_000129.', 'ONCO_HP20002450_000132.', 'not_tumor_HP19.4075_000136.jpg', 'pRCC_HP18.5818_000141.jpg', 'ccRCC_HP12.13588_000145.jpg', 'not_t

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TRUE LABEL PER PAZIENTE {'HP20002450': 'ONCO', 'HP18014084': 'CHROMO', 'HP12.7726': 'pRCC', 'HP11.12318': 'ccRCC', 'HP19.1277': 'pRCC', 'HP19.4075': 'ccRCC', 'HP18.5818': 'pRCC', 'HP14.4279': 'pRCC', 'HP12.13588': 'ccRCC', 'HP18.13618': 'pRCC', 'HP12.6073': 'ccRCC', 'HP19.8394': 'ccRCC', 'HP19.10064': 'ccRCC', 'HP02.10180': 'ccRCC', 'HP12.6691': 'ccRCC', 'HP12.8355': 'ccRCC', 'HP19.7421': 'ccRCC', 'HP15.12550': 'ccRCC'}

📊 Risultati Majority Voting (paziente-level):
              precision    recall  f1-score   support

      CHROMO       0.00      0.00      0.00         1
        ONCO       0.00      0.00      0.00         1
       ccRCC       1.00      0.27      0.43        11
        pRCC       0.33      1.00      0.50         5

    accuracy                           0.44        18
   macro avg       0.33      0.32      0.23        18
weighted avg       0.70      0.44      0.40        18

📉 Confusion Matrix:
[[0 0 0 1]
 [0 0 0 1]
 [0 0 3 8]
 [0 0 0 5]]
✅ Totale pazienti classificat

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m